<a href="https://colab.research.google.com/github/antbartash/max_temp/blob/main/data_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tqdm

In [4]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data2.csv'
data = pd.read_csv(data_path, index_col=0)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

print(data.shape)
data.head()

(40898, 4)


,STATION,NAME,DATE,TMAX
0,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-01,12.2
1,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-02,10.6
2,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-03,8.3
3,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-04,6.1
4,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-05,6.1


In [5]:
data_features = pd.DataFrame()

for station in tqdm.tqdm(data['STATION'].unique()):
  data_station = data[data['STATION'] == station].copy()
  data_station = data_station.sort_values('DATE').reset_index(drop=True)
  for id in data_station.index.values:
    if id >= 15:
      data_new_row = pd.DataFrame({
          'STATION': data_station.loc[id, 'STATION'],
          'NAME': data_station.loc[id, 'NAME'],
          'DATE': data_station.loc[id, 'DATE'],
          'TARGET': data_station.loc[id, 'TMAX'],
          'TMAX_d1': data_station.loc[id-1, 'TMAX'],
          'TMAX_d1_d2_diff': data_station.loc[id-1, 'TMAX'] - data_station.loc[id-2, 'TMAX']
      }, index=[0])
      for lag in range(1, 15):
          data_new_row[f'TMAX_d{lag}'] = data_station.loc[id-lag, 'TMAX']
      for window in [3, 5, 7, 10, 14]:
        for agg_func in ['mean', 'std', 'min', 'max', 'median']:
          data_new_row[f'TMAX_{window}day_{agg_func}'] = data_station.loc[id-window:id-1, 'TMAX'].agg(agg_func)
      for window in [3, 5, 7, 10, 14]:
        data_new_row[f'TMAX_{window}day_mean_diff'] = np.mean(data_station.loc[id-window:id-1, 'TMAX'].values - data_station.loc[id-window-1:id-2, 'TMAX'].values)
      data_features = pd.concat([data_features, data_new_row], ignore_index=True)

print(data_features.shape)
data_features.head()

100%|██████████| 8/8 [15:39<00:00, 117.43s/it]

(40778, 49)


,STATION,NAME,DATE,TARGET,TMAX_d1,TMAX_d1_d2_diff,TMAX_d2,TMAX_d3,TMAX_d4,TMAX_d5,...,TMAX_14day_mean,TMAX_14day_std,TMAX_14day_min,TMAX_14day_max,TMAX_14day_median,TMAX_3day_mean_diff,TMAX_5day_mean_diff,TMAX_7day_mean_diff,TMAX_10day_mean_diff,TMAX_14day_mean_diff
0,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-16,15.6,15.0,-1.1,16.1,10.0,14.4,9.4,...,9.278571,4.868925,1.7,16.7,8.85,0.200000,2.12,1.900000,0.89,0.200000
1,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-17,14.4,15.6,0.6,15.0,16.1,10.0,14.4,...,9.635714,5.148653,1.7,16.7,8.85,1.866667,1.24,1.828571,0.73,0.357143
2,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-18,17.2,14.4,-1.2,15.6,15.0,16.1,10.0,...,10.071429,5.283272,1.7,16.7,9.70,-0.566667,0.00,1.428571,-0.23,0.435714
3,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-19,21.1,17.2,2.8,14.4,15.6,15.0,16.1,...,10.864286,5.470987,1.7,17.2,12.20,0.733333,1.44,1.114286,1.55,0.792857
4,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-20,23.9,21.1,3.9,17.2,14.4,15.6,15.0,...,11.935714,5.916809,1.7,21.1,14.40,1.833333,1.00,0.957143,1.83,1.071429


In [6]:
data_features['MONTH'] = data_features['DATE'].dt.month.values

encoder = OneHotEncoder()
encoder.fit(data_features[['STATION']])
stations = pd.DataFrame.sparse.from_spmatrix(encoder.transform(data_features[['STATION']]), columns=encoder.get_feature_names_out())
data_features = pd.concat([data_features, stations], axis=1)

encoder = OneHotEncoder()
encoder.fit(data_features[['MONTH']])
months = pd.DataFrame.sparse.from_spmatrix(encoder.transform(data_features[['MONTH']]), columns=encoder.get_feature_names_out())
data_features = pd.concat([data_features, months], axis=1)

print(data_features.shape)
data_features.head()

(40778, 70)


,STATION,NAME,DATE,TARGET,TMAX_d1,TMAX_d1_d2_diff,TMAX_d2,TMAX_d3,TMAX_d4,TMAX_d5,...,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12
0,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-16,15.6,15.0,-1.1,16.1,10.0,14.4,9.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-17,14.4,15.6,0.6,15.0,16.1,10.0,14.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-18,17.2,14.4,-1.2,15.6,15.0,16.1,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-19,21.1,17.2,2.8,14.4,15.6,15.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,USW00012916,"NEW ORLEANS AIRPORT, LA US",2010-01-20,23.9,21.1,3.9,17.2,14.4,15.6,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_features.drop(columns=['STATION', 'NAME']).to_csv('data_features_w_base.csv', index=False)